<a href="https://colab.research.google.com/github/sum-coderepo/Git-Test/blob/main/SentenceEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import json
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import re
year_pattern = r'([1-2][0-9]{3})'

In [ ]:
df = pd.read_csv('/content/sample_data/test.csv')
df.head(10)

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

# model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_ckpt = "sentence-transformers/all-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

In [ ]:
!pip install transformers